In [1]:
# imports and setup
from bs4 import BeautifulSoup  
import pandas as pd
import requests
import datetime

In [2]:
##import player_list.csv file and build df_player_list dataframe##
df_player_list = pd.read_csv("player_list.csv")

In [3]:
date_of_birth=list()
place_of_birth=list()
age=list()
height=list()
citizenship=list()
position=list()
foot=list()
current_club=list()
joined_date=list()
current_market_value=list()
highest_market_value=list()
temp=list()
x=0
place_of_birth_flag=False
height_flag=False
foot_flag=False

In [4]:
##A loop that runs on each player page, saving in a dataframe the player information (age, height, citizenship, etc.)##
for link in df_player_list['player link']:
    html = requests.get(link,headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'})
    soup = BeautifulSoup(html.content, "html.parser")
    response=soup("table" , attrs={"class": "auflistung"})[0]
    for trs in response("tr"):
        if trs.th.text.strip() == "Date of birth:":
            date_of_birth.append(trs.td.text.strip())
            #####
        if trs.th.text.strip() == "Place of birth:":
            place_of_birth.append(trs.td.text.strip())
            place_of_birth_flag=True
        elif ((trs.th.text.strip() != "Place of birth:") and (x > 2) and (place_of_birth_flag == False)):
            place_of_birth.append("NaN")
            place_of_birth_flag=True
            #####
        if trs.th.text.strip() == "Age:":
            age.append(trs.td.text.strip())
            #####
        if trs.th.text.strip() == "Height:":
            height.append(trs.td.text.strip().split()[0])
            height_flag=True
        elif ((trs.th.text.strip() != "Height:") and (x > 4) and (height_flag == False)):
            height.append("NaN")
            height_flag=True
            #####
        if trs.th.text.strip() == "Citizenship:":
            citizenship.append(trs.td.text.strip())
        if trs.th.text.strip() == "Position:":
            position.append(trs.td.text.strip())
            #####
        if trs.th.text.strip() == "Foot:":
            foot.append(trs.td.text.strip())
            foot_flag=True
        elif ((trs.th.text.strip() != "Foot:") and (x > 7) and (foot_flag == False)):
            foot.append("NaN")
            foot_flag=True
            #####
        if trs.th.text.strip() == "Current club:":
            current_club.append(trs.td.text.strip())
        if trs.th.text.strip() == "Joined:":
            joined_date.append(trs.td.text.strip())
        x=x+1
    x=0
    place_of_birth_flag=False
    height_flag=False
    foot_flag=False
    response=soup("div" , attrs={"class": "large-5 columns small-12"})[0]
    for market_value in response("div" , attrs={"class": "right-td"}):
        temp.append(market_value.get_text().strip().split()[0])
    current_market_value.append(temp[0])
    highest_market_value.append(temp[2])
    temp.clear()


In [20]:
##Building df_player_data dataframe from the lists above##
df_player_data = pd.DataFrame({"date_of_birth":date_of_birth,
                               "place_of_birth":place_of_birth,
                               "age":age,
                               "height":height,
                               "citizenship":citizenship,
                               "position":position,
                               "foot":foot,
                               "current_club":current_club,
                               "joined_date":joined_date,
                               "current value-EUR":current_market_value,
                               "highest value-EUR":highest_market_value})


In [21]:
df_player_data

,date_of_birth,place_of_birth,age,height,citizenship,position,foot,current_club,joined_date,current value-EUR,highest value-EUR
0,"Dec 20, 1998",Paris,22,"1,78",France,attack - Centre-Forward,right,Paris Saint-Germain,"Jul 1, 2018",€160.00m,€200.00m
1,"Jul 21, 2000",Leeds,20,"1,94",Norway,attack - Centre-Forward,left,Borussia Dortmund,"Jan 1, 2020",€130.00m,€130.00m
2,"Mar 25, 2000",London,21,"1,80",England,attack - Right Winger,right,Borussia Dortmund,"Aug 31, 2017",€100.00m,€130.00m
3,"May 12, 1997",Arkel,24,"1,80",Netherlands,midfield - Central Midfield,right,FC Barcelona,"Jul 1, 2019",€90.00m,€90.00m
4,"Sep 8, 1994",Maia,26,"1,79",Portugal,midfield - Attacking Midfield,right,Manchester United,"Jan 29, 2020",€90.00m,€90.00m
...,...,...,...,...,...,...,...,...,...,...,...
8208,"Jan 7, 1989","Tembisa, Gauteng",32,"1,76",South Africa,midfield - Central Midfield,NaN,Mamelodi Sundowns FC,"Jul 1, 2020",€525Th.,€850Th.
8209,"Aug 6, 1996",Napoli,24,"1,82",Italy,midfield - Central Midfield,NaN,Ternana Calcio,"Oct 5, 2020",€650Th.,€650Th.
8210,"May 15, 1986","Great River, New York",35,"1,85",United States,Goalkeeper,NaN,Sporting Kansas City,"Aug 13, 2014",€600Th.,€1.00m
8211,"Oct 12, 1992",Lima,28,"1,77",Peru,midfield - Attacking Midfield,NaN,Club Sporting Cristal,"Jan 1, 2019",€575Th.,€750Th.


In [22]:
##Export the dataframe to CSV file###
df_player_data.to_csv("player_data.csv")